In [28]:
import pyparsing
from speakeasypy import Speakeasy
import rdflib
from rdflib.namespace import split_uri
from rdflib.namespace import Namespace, RDF, RDFS, XSD
import pickle

In [2]:
graph = rdflib.Graph()
graph.parse("dataset/14_graph.nt", format="turtle")

<Graph identifier=Nf7c6d06742a64776b33829c998a0244a (<class 'rdflib.graph.Graph'>)>

In [42]:
predicates = list(graph.predicates())

In [23]:
with open("predicates.pkl", "wb") as f:
    pickle.dump(predicates, f)

In [25]:
with open("predicates.pkl", "rb") as f:
    loaded_predicates = pickle.load(f)

print(loaded_predicates)

[rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P18'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P161'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P161'), rdflib.term.URIRef('http://schema.org/description'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P161'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P3216'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1441'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P50'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P175'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1346'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P5021'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P161'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P58'), rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P136'), rdflib.term.URIRef('http://www.wikidata.org/pro

In [ ]:
result = []
for uri in predicates:
    property_id = split_uri(uri)[1]
    
    query = f"""
        PREFIX ddis: <http://ddis.ch/atai/> 
        PREFIX wd: <http://www.wikidata.org/entity/> 
        PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
        PREFIX schema: <http://schema.org/> 
        SELECT ?propertyLabel WHERE {{
          wdt:{property_id} rdfs:label ?propertyLabel .
          FILTER(LANG(?propertyLabel) = "en")
        }}
        LIMIT 1
        """
    
    res = graph.query(query)
    result.append([str(s) for s, in res])
print(result)